In [1]:
%reset
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

## read data 
CIP_data = pd.read_csv("CIP_data_encode_prev.csv")


Nothing done.


In [27]:
X = CIP_data[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]

y = CIP_data['Susceptible']

model = RandomForestClassifier(n_estimators = 100, min_samples_leaf=5, class_weight='balanced')
model_fit = model.fit(X, y)

print(model_fit.score(X,y)) #0.7013699360815028

y_predict = model_fit.predict(X)

ROC_AUC_random_forest = metrics.roc_auc_score(y, y_predict) ## 0.756384214489288
print(ROC_AUC_random_forest)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y, y_predict)) ## but get model accuracy of 0.7013699360815028



0.702863441997742
0.7564421188563271
ACCURACY OF THE MODEL:  0.702863441997742
0.7563471162721497
ACCURACY OF THE MODEL:  0.7023922764408331
0.7564421188563271
ACCURACY OF THE MODEL:  0.7023922764408331


In [12]:

n_iterations = 10
bootstrapped_stats = []

for i in range(n_iterations):
       #sample = resample(CIP_data, replace=True, n_samples=len(CIP_data)) ##(a) sample n individuals with replacement
       sample = CIP_data.sample(frac = 1, replace=True, random_state = 1) ##(a) sample n individuals with replacement

       X_sample  = sample[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]

       y_sample = sample['Susceptible']

       model = RandomForestClassifier(n_estimators = 100, min_samples_leaf=5, class_weight='balanced') #calculate APPARENT performance - ROC
       model_sample = model.fit(X_sample, y_sample)
       y_predict_sample = model_sample.predict(X_sample) 
       ROC_AUC_random_forest_sample = metrics.roc_auc_score(y_sample, y_predict_sample)

       y_test = model_sample.predict(X) #performance on original data  
       ROC_AUC_random_forest_test = metrics.roc_auc_score(y_sample, y_test)

       optomisation = ROC_AUC_random_forest_sample - ROC_AUC_random_forest_test #optimisation

       bootstrapped_stats.append(
        {
            'Sample ROC': ROC_AUC_random_forest_sample,
            'Test ROC': ROC_AUC_random_forest_test,
            'Optimisation': optomisation
        }
       )



In [13]:
bootstrapped_stats = pd.DataFrame(bootstrapped_stats)
print(bootstrapped_stats.head())
## Step 3: Get average optimization

average_optimisation = bootstrapped_stats["Optimisation"].mean() 

## Step 4: Get optimization-corrected performance

optimization_corrected_performance = ROC_AUC_random_forest - average_optimisation ##

print(optimization_corrected_performance)

## get CI 

conf_interval = np.percentile(bootstrapped_stats["Optimisation"],[2.5,97.5])
Upper_bootstrap_CI = optimization_corrected_performance +conf_interval[0]
Lower_bootstrap_CI = optimization_corrected_performance - conf_interval[1]

print(Upper_bootstrap_CI, Lower_bootstrap_CI)

   Sample ROC  Test ROC  Optimisation
0    0.763819  0.494129      0.269689
1    0.763894  0.493990      0.269904
2    0.763786  0.494677      0.269109
3    0.763913  0.493935      0.269978
4    0.763923  0.493500      0.270423
0.4865643228777631
0.7558039188131078 0.21620475522397908


In [22]:
## Random search https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

X = CIP_data[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]

y = CIP_data['Susceptible']

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)

space = dict()
space['n_estimators'] = np.arange(0, 200, 10)
space['max_features'] = ['sqrt']
#space['max_depth'] = np.arange(0, 200, 10)
space['min_samples_split'] = [2,5,10]
#space['min_samples_leaf'] = [1,2,4]
model = RandomForestClassifier(class_weight = 'balanced', max_features='sqrt', min_samples_leaf = 3)
model_fit = model.fit(X, y)

search = RandomizedSearchCV(model, space, n_iter=10, scoring='roc_auc', n_jobs=-1, cv=cv, random_state=1)
result = search.fit(X, y)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
#Best Score: 0.8125469650495328 from cross validation
#Best Hyperparameters: {'n_estimators': 150, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto'}

/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
100 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 341, in fit
    self._validate_params()
  File "/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/base.py", line 570, i

Best Score: 0.8119051266148526
Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 10, 'max_features': 'sqrt'}


In [41]:
#### now try bootstrapping with new hyperparameters
#bootstrap data
n_iterations = 10
bootstrapped_stats = []

#1. Create model using all data and get ROC_AUC ("ROC_AUC_random_forest")
X = CIP_data[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]
y = CIP_data['Susceptible']

model = RandomForestClassifier(n_estimators = 100, min_samples_leaf=5, class_weight='balanced')
model_fit = model.fit(X, y)
y_predict = model_fit.predict(X)

ROC_AUC_random_forest_apparent = metrics.roc_auc_score(y, y_predict)

for i in range(n_iterations):
       #2. (A) Sample all individuals w/replacement
        sample = CIP_data.sample(frac = 1, replace=True) ##(a) sample n individuals with replacement
        X_sample = sample[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]
        y_sample = sample['Susceptible']


       #  (B) Develop predictive model and find apparent performance
        model_fit = model.fit(X_sample, y_sample)
        y_sample_predict = model_fit.predict(X_sample)
        ROC_AUC_random_forest_bootstrap_sample_performance = metrics.roc_auc_score(y_sample, y_sample_predict) 

       #  (C) Performance of predictive model on original sample (i.e. original population, X)
        y_test_predict = model_fit.predict(X)
        ROC_AUC_random_forest_bootstrap_test_performance = metrics.roc_auc_score(y, y_test_predict) ## 0.756384214489288
      ### (D) Calculate optimisation by getting (B) - (D) 
        optimism = ROC_AUC_random_forest_bootstrap_sample_performance - ROC_AUC_random_forest_bootstrap_test_performance



        bootstrapped_stats.append(
        {
            'Sample ROC': ROC_AUC_random_forest_bootstrap_sample_performance,
            'Test ROC': ROC_AUC_random_forest_bootstrap_test_performance,
            'Optimisation': optimism
        }
       )


bootstrapped_stats = pd.DataFrame(bootstrapped_stats)
print(bootstrapped_stats.head())
## Step 3: Get average optimization

average_optimisation = bootstrapped_stats["Optimisation"].mean() 

## Step 4: Get optimization-corrected performance

optimization_corrected_performance = ROC_AUC_random_forest_apparent - average_optimisation ##

print(optimization_corrected_performance)

## get CI 

conf_interval = np.percentile(bootstrapped_stats["Optimisation"],[2.5,97.5])
Upper_bootstrap_CI = optimization_corrected_performance +conf_interval[0]
Lower_bootstrap_CI = optimization_corrected_performance - conf_interval[1]

print(Upper_bootstrap_CI, Lower_bootstrap_CI)

   Sample ROC  Test ROC  Optimisation
0    0.763100  0.750028      0.013072
1    0.761132  0.749185      0.011947
2    0.764335  0.748335      0.016000
3    0.760371  0.748950      0.011421
4    0.762243  0.748656      0.013586
0.7430470754620264
0.7533947962515627 0.7269995165810333


In [37]:
### No loop bootstrap 

## bootstrap 
#1. Create model using all data and get ROC_AUC ("ROC_AUC_random_forest")
X = CIP_data[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]
y = CIP_data['Susceptible']

model = RandomForestClassifier(n_estimators = 100, min_samples_leaf=5, class_weight='balanced')
model_fit = model.fit(X, y)

y_predict = model_fit.predict(X)

ROC_AUC_random_forest_apparent = metrics.roc_auc_score(y, y_predict) ## 0.756384214489288
print(ROC_AUC_random_forest_apparent)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y, y_predict)) ## but get model accuracy of 0.7013699360815028

#2. (A) Sample all individuals w/replacement
sample = CIP_data.sample(frac = 1, replace=True, random_state = 1) ##(a) sample n individuals with replacement
X_sample = sample[['MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC']]
y_sample = sample['Susceptible']

#   (B) Develop predictive model and find apparent performance
model_fit = model.fit(X_sample, y_sample)
y_sample_predict = model_fit.predict(X_sample)
ROC_AUC_random_forest_bootstrap_sample_performance = metrics.roc_auc_score(y_sample, y_sample_predict) ## 0.756384214489288
print(ROC_AUC_random_forest_bootstrap_sample_performance)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_sample, y_sample_predict)) ## but get model accuracy of 0.7013699360815028

#   (C) Performance of predictive model on original sample (i.e. original population, X)
y_test_predict = model_fit.predict(X)
ROC_AUC_random_forest_bootstrap_test_performance = metrics.roc_auc_score(y, y_test_predict) ## 0.756384214489288
print(ROC_AUC_random_forest_bootstrap_test_performance)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_sample, y_sample_predict)) ## but get model accuracy of 0.7013699360815028

### (D) Calculate optimisation by getting (B) - (D) 
optimism = ROC_AUC_random_forest_bootstrap_sample_performance - ROC_AUC_random_forest_bootstrap_test_performance
print("OPTIMISATION: ", ROC_AUC_random_forest_bootstrap_sample_performance - ROC_AUC_random_forest_bootstrap_test_performance)


### 6. Optimisation-corrected performance
print(ROC_AUC_random_forest_apparent - optimism)

0.7565752294989667
ACCURACY OF THE MODEL:  0.7029167814947505
0.7639643250818674
ACCURACY OF THE MODEL:  0.7091575026447501
0.7503941749717263
ACCURACY OF THE MODEL:  0.7091575026447501
OPTIMISATION:  0.013570150110141066
0.7430050793888257
